Copyright (c) 2023, University of California, Merced. All rights reserved.

This file is part of the MummyNutsBench software package developed by
the team members of Prof. Xiaoyi Lu's group (PADSYS Lab) at the University
of California, Merced.

For detailed copyright and licensing information, please refer to the license
file LICENSE in the top level directory.

# Mummy Nut Detection Model

A very helpful guide to using YOLOv5 is [here](https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data).

In [1]:
#@markdown GPU type
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-28c01018-86ae-5030-7618-f878c603c985)


Cloning the git repo for YOLOv5 

Apparently it's a [bug](https://github.com/ultralytics/yolov5/issues/7127) that appeared only a few days ago.


In [ ]:
!git clone https://github.com/ultralytics/yolov3.git
!python3 -m pip install -r yolov3/requirements.txt 

Download fullres dataset

In [4]:
!mkdir -p datasets/MNut2
%cd datasets/MNut2
!rm -rf *
!wget -O MNut2.zip wget 'https://drive.google.com/uc?id=1Sz_O5Z3hADj8lMnAR3knJCCkWwzX3SHT&export=download&confirm=t'
!unzip MNut2.zip
!mv Test_2/* .;
!rm -rf Test_2; rm MNut2.zip
# !curl -L "https://public.roboflow.com/ds/rw2qoOJbXa?key=4TwfyBeUFo" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
%cd ../..

/content/datasets/MNut2
--2022-05-02 16:05:25--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2022-05-02 16:05:25--  https://drive.google.com/uc?id=1Sz_O5Z3hADj8lMnAR3knJCCkWwzX3SHT&export=download&confirm=t
Resolving drive.google.com (drive.google.com)... 173.194.196.100, 173.194.196.113, 173.194.196.139, ...
Connecting to drive.google.com (drive.google.com)|173.194.196.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7kohqrdjdsnpsovs9jtrq6j305cnhkdj/1651507500000/13385269720894965035/*/1Sz_O5Z3hADj8lMnAR3knJCCkWwzX3SHT?e=download [following]
--2022-05-02 16:05:25--  https://doc-04-64-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7kohqrdjdsnpsovs9jtrq6j305cnhkdj/1651507500000/13385269720894965035/*/1Sz_O5Z3hADj8lMnAR3knJCCkWwzX3SHT?e=download
Resolving doc-

In [22]:
with open('datasets/MNut2/data.yaml', 'w') as f:
  f.write("""train: datasets/MNut2/train/\nval: datasets/MNut2/valid/\n\nnc: 1\nnames: ['nut']""")

In [23]:
!cat 'datasets/MNut2/data.yaml'

train: datasets/MNut2/train/
val: datasets/MNut2/valid/

nc: 1
names: ['nut']

In [24]:
!wandb disabled
!python3 yolov3/train.py --rect --batch 16 --epochs 200 --data datasets/MNut2/data.yaml --weights yolov3.pt

W&B disabled.
train: weights=yolov3.pt, cfg=, data=datasets/MNut2/data.yaml, hyp=yolov3/data/hyps/hyp.scratch.yaml, epochs=200, batch_size=16, imgsz=640, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=yolov3/runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov3 ✅
YOLOv3 🚀 v9.6.0-16-gd58ba5e torch 1.11.0+cu113 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, de

## Validation of the Model

In [25]:
import os
p = "yolov3/runs/train"
latest = max(os.listdir(p), key=lambda run: os.stat(os.path.join(p, run)).st_ctime)
weights = os.path.join(p, latest, "weights/best.pt")

# !python3 yolov5/val.py --weights $weights --data datasets/raccoon/data.yaml --img $imgsize

In [ ]:
from IPython.display import Image, display

!python3 yolov3/detect.py --weights $weights --source datasets/MNut2/test/IMG_4445_jpg.rf.dc087c9f6c74bb702ff05635335c5a35.jpg --conf-thres 0.05 --hide-labels
p = "yolov3/runs/detect"
latest = max(os.listdir(p), key=lambda run: os.stat(os.path.join(p, run)).st_ctime)
file_one = os.listdir(os.path.join(p, latest))[0]
p = os.path.join(p, latest, file_one)
display(Image(p))

### Results

In [26]:
from IPython.display import Image, display
import os
p = "yolov3/runs/train"
p = os.path.join(p, "../train")
latest = max(os.listdir(p), key=lambda run: os.stat(os.path.join(p, run)).st_ctime)
result_path = os.path.join(p, latest, "results.csv")
!cat $result_path
# images = [os.path.join(p, latest, x) for x in os.listdir(os.path.join(p, latest))]
# for im in images:
#     display(Image(filename=im))

               epoch,      train/box_loss,      train/obj_loss,      train/cls_loss,   metrics/precision,      metrics/recall,     metrics/mAP_0.5,metrics/mAP_0.5:0.95,        val/box_loss,        val/obj_loss,        val/cls_loss,               x/lr0,               x/lr1,               x/lr2
                   0,             0.19847,            0.023863,                   0,            0.005028,            0.014085,          0.00032101,          3.2101e-05,             0.19868,            0.021552,                   0,               1e-05,               1e-05,             0.09991
                   1,             0.19993,            0.023739,                   0,                   0,                   0,                   0,                   0,             0.19822,            0.021258,                   0,          2.9998e-05,          2.9998e-05,             0.09973
                   2,             0.19874,            0.023072,                   0,                   0,             

## Downloading the model

In [ ]:
from google.colab import files
files.download(weights)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Exporting Model Weights

In [ ]:
!python3 yolov3/export.py --include coreml --weights ~/Downloads/best.pt